# Gmail Newsletter Exploration

### Goal
- explore the features of the Gmail api 
- gather relevant newsletter mails form the mailbox
- scrape newsletter based on thier Design so that i can standardize the database article mentioned in the mails

### DB-Layout
- id
- url
- content name
- type of content
- full text or snippet of content

In [9]:
# imports
from google.oauth2.credentials import Credentials   # handel auth
from googleapiclient.discovery import build         # used to make the queries

import os                  # handle paths
import base64                   # decrypt mail data
from bs4 import BeautifulSoup   # parse data after decryption
import pickle
import datetime

relevant paramters for list()
- maxResult: default 100 / up to 500
- q: only returns messages matching a query "from:lon@dataelixir.com"     
- pageToken: "retrive a specific page of result in a list"

In [2]:
# Define SCOPES
# relevant for auth
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

pathToToken = '/Users/markusmuller/python/projects/gmail-newsletter-db/token.json'

creds = Credentials.from_authorized_user_file(pathToToken, SCOPES)

# connect to Gmail api
service = build('gmail', 'v1', credentials=creds)

# request a list of all the messages
result = service.users().messages().list(userId='markus.mueller.ds@gmail.com', y).execute()

In [5]:
print(type(result))
# frirst 10 result of 
print(result)
print('Next page Token: ',  result['nextPageToken'])
print('length: ', len(result['messages']))

<class 'dict'>
{'messages': [{'id': '17bf36ad1f8ffa14', 'threadId': '17bf36ad1f8ffa14'}, {'id': '17bf2d743a2a4a64', 'threadId': '17bf2d743a2a4a64'}, {'id': '17bf2ba112b2990e', 'threadId': '17bf2ba112b2990e'}, {'id': '17bf2b9a28a4e1cf', 'threadId': '17bf2b9a28a4e1cf'}, {'id': '17bed324d52f36ee', 'threadId': '17bed324d52f36ee'}, {'id': '17beb2ea7a06eb5e', 'threadId': '17beb2ea7a06eb5e'}, {'id': '17be8bcdaa338b35', 'threadId': '17be8bcdaa338b35'}, {'id': '17be5f07abd13e84', 'threadId': '17be5f07abd13e84'}, {'id': '17be5ac70eaf3f1d', 'threadId': '17be5ac70eaf3f1d'}, {'id': '17be368851c51489', 'threadId': '17be368851c51489'}], 'nextPageToken': '17088860671368477437', 'resultSizeEstimate': 11}
Next page Token:  17088860671368477437
length:  10


Now we can use the ids to get the content of the mails

In [4]:
msg_ids = result.get('messages')

In [47]:
for msg in msg_ids:
    print(msg['id'])

17bf36ad1f8ffa14
17bf2d743a2a4a64
17bf2ba112b2990e
17bf2b9a28a4e1cf
17bed324d52f36ee
17beb2ea7a06eb5e
17be8bcdaa338b35
17be5f07abd13e84
17be5ac70eaf3f1d
17be368851c51489
17be067784b98401
17bdf62d721058f5
17bdafc60ea3954c
17bd51b3bf685077
17bd4d4f6a8e5bd7
17bd41ed7e3689c9
17bd157766588699
17bd053251a031b3
17bcf5e615f87ecd
17bcef38c960fc3b
17bca0bc853a32f3
17bc92883487c59a
17bc6bf1817afc13
17bc4a99883075e1
17bc47ca455dea43
17bc1e3e4c595966
17bc17764ecf3708
17bbf853dfebce27
17bba2e6ee21ba8a
17bb7432c5ae73c8
17bb6870bb68b786
17bb0770b57f10ee
17bad571fe5322f8
17bab51d2e5a93ff
17bab5189de5ff8a
17bab32fc35f68e2
17ba5c0f63e5d7a6
17ba598e729bf32b
17ba349206733d4b
17ba27ef25924242
17ba0a53c538f36a
17b9de70877d3d32
17b9dd5d1c159068
17b9d97f81a5b755
17b9d3e197b14d4f
17b8c4228e41efa3
17b8c0b2c93af7d1
17b890045309533f
17b880171729f007
17b87457879bb57f
17b86ab90b36ee35
17b829a3d641cc67
17b7f2b2004c9889
17b7c8c29c863548
17b7c536cd4056ff
17b79c95aed7e0a6
17b797edd35042ff
17b78787b99689b8
17b77a03091643

In [5]:
# get the data for one mail
id_ = '17bed324d52f36ee'
txt = service.users().messages().get(userId='me', id=id_).execute()


message object https://developers.google.com/gmail/api/reference/rest/v1/users.messages#Message:
- id: ID of the message
- threadId: The ID of the thread the message belongs to. 
- labelIds: List of IDs of labels applied to this message.
- snippet: A short part of the message text.
- historyId: The ID of the last history record that modified this message.
- internalDate: The internal message creation timestamp (epoch ms)
- payload: The parsed email structure in the message parts. (here is the relevant data)

In [6]:
print(txt.keys())

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])


In [7]:
payload = txt['payload']

Next cell saves the output in an external file os I can view the content 

In [42]:
%%capture cap --no-stderr
print(payload)
with open('test.txt', 'w') as f:
    f.write(str(cap))

In [8]:
payload.keys()

dict_keys(['partId', 'mimeType', 'filename', 'headers', 'body', 'parts'])

In [9]:
print(payload['headers'])

[{'name': 'Delivered-To', 'value': 'markus.mueller.ds@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6e04:139c:0:0:0:0 with SMTP id be28csp2135701imb;        Wed, 15 Sep 2021 23:01:42 -0700 (PDT)'}, {'name': 'X-Google-Smtp-Source', 'value': 'ABdhPJyxdMCapZIZ5Y2skh15iwb4zjdvYJLLsM4QvWJA0SJ0kRLSxC91nVNXVGAVCOPtoTAajtQ2'}, {'name': 'X-Received', 'value': 'by 2002:a05:6902:110d:: with SMTP id o13mr4912149ybu.530.1631772101846;        Wed, 15 Sep 2021 23:01:41 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1631772101; cv=none;        d=google.com; s=arc-20160816;        b=BQI6VxIPo295XbyRQYuNmSHaNs3p/jg82bkgLzhoIwVuSh9IfN+IQdEmXONgWqVhbJ         A8gtVP29JeiD2nEpEd+j/jQoiHlPowC2/1v+JIn3XcVIWKvu/Yzade4fEnGRjNOiFfMC         QNjRNBYEmW0GTwEMtEFYRZWjKf0btWjR+igqfKaPhv/4CekxQU7IdnC/mewIJeTYDsX1         h0SWLsDTwqVy5XHJUy8xszprwtvJCWqSmfpLK7r+EL1V3PuBzc0EzUunYoyjTOA3I9Tk         9t68bAN+Gc3Vo9IWsYLod+gJj+kpM550alSh+QQgA5CqbtbN9pilltmlE4+tyJkES/WO         qKzg=='}, {'na

In [11]:
for item in payload['headers']:
    print(item)
    print('-'*30)


{'name': 'Delivered-To', 'value': 'markus.mueller.ds@gmail.com'}
------------------------------
{'name': 'Received', 'value': 'by 2002:a05:6e04:139c:0:0:0:0 with SMTP id be28csp2135701imb;        Wed, 15 Sep 2021 23:01:42 -0700 (PDT)'}
------------------------------
{'name': 'X-Google-Smtp-Source', 'value': 'ABdhPJyxdMCapZIZ5Y2skh15iwb4zjdvYJLLsM4QvWJA0SJ0kRLSxC91nVNXVGAVCOPtoTAajtQ2'}
------------------------------
{'name': 'X-Received', 'value': 'by 2002:a05:6902:110d:: with SMTP id o13mr4912149ybu.530.1631772101846;        Wed, 15 Sep 2021 23:01:41 -0700 (PDT)'}
------------------------------
{'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1631772101; cv=none;        d=google.com; s=arc-20160816;        b=BQI6VxIPo295XbyRQYuNmSHaNs3p/jg82bkgLzhoIwVuSh9IfN+IQdEmXONgWqVhbJ         A8gtVP29JeiD2nEpEd+j/jQoiHlPowC2/1v+JIn3XcVIWKvu/Yzade4fEnGRjNOiFfMC         QNjRNBYEmW0GTwEMtEFYRZWjKf0btWjR+igqfKaPhv/4CekxQU7IdnC/mewIJeTYDsX1         h0SWLsDTwqVy5XHJUy8xszprwtvJCWqSmfpLK7r+EL1V3PuBz

In [14]:
print('recived: ',  payload['headers'][1].get('value'))



recived:  by 2002:a05:6e04:139c:0:0:0:0 with SMTP id be28csp2135701imb;        Wed, 15 Sep 2021 23:01:42 -0700 (PDT)


In [19]:
# payload['headers'] is a list of dicts
for d in payload['headers']:
    if d['name'] == 'Date':
        date = d['value']
    if d['name'] == 'From':
        sender = d['value']
    if d['name'] == 'Subject':
        subject = d['value']
    
print('date: ', date)
print('from: ', sender)
print('subject: ', subject)

date:  Thu, 16 Sep 2021 05:59:57 +0000
from:  KDnuggets <editor1@kdnuggets.com>
subject:  Intro to Data Labeling for Machine Learning


In [24]:
parts = payload['parts'][0]


In [30]:
# get data of the messages
data = parts['body']['data']
data = data.replace("-","+").replace("_","/")

# data is encoded
decoded_data = base64.b64decode(data)

# parse obtained lxml with BeautifulSoup 
soup = BeautifulSoup(decoded_data , "lxml")
body = soup.body()

In [32]:
type(body)

bs4.element.ResultSet

In [35]:
body

[<p>https://kdnuggets.us12.list-manage.com/track/click?u=4f2891ebb155b23f120ece0bd&amp;id=6226337bb4&amp;e=bfdfd73c17
 
 Once upon a time, there was a company that dreamed of changing the world with machine learning. The brilliant data team labeled away day and night but it was never enough to data. The models just weren't performing correctly. They longed for the blissful days of data analysis and innovation.
 
 Sound familiar? You might be ready to outsource your data labeling.
 
 Your dream of changing the world can come true with a little help from some strangers across the globe. If this is your first time working with outsourced teams, it might seem daunting. This free guide (https://kdnuggets.us12.list-manage.com/track/click?u=4f2891ebb155b23f120ece0bd&amp;id=b2ae45d703&amp;e=bfdfd73c17) will help you confidently start that part of your innovation journey.
 • Tips for successfully outsourcing AI and ML projects and ensuring quality datasets
 • Common data labeling terminology, t

In [33]:
body_str = str(body)

In [34]:
body_str

"[<p>https://kdnuggets.us12.list-manage.com/track/click?u=4f2891ebb155b23f120ece0bd&amp;id=6226337bb4&amp;e=bfdfd73c17\r\n\r\nOnce upon a time, there was a company that dreamed of changing the world with machine learning. The brilliant data team labeled away day and night but it was never enough to data. The models just weren't performing correctly. They longed for the blissful days of data analysis and innovation.\r\n\r\nSound familiar? You might be ready to outsource your data labeling.\r\n\r\nYour dream of changing the world can come true with a little help from some strangers across the globe. If this is your first time working with outsourced teams, it might seem daunting. This free guide (https://kdnuggets.us12.list-manage.com/track/click?u=4f2891ebb155b23f120ece0bd&amp;id=b2ae45d703&amp;e=bfdfd73c17) will help you confidently start that part of your innovation journey.\r\n• Tips for successfully outsourcing AI and ML projects and ensuring quality datasets\r\n• Common data labeli

## Test saving and loading query result as pkl-file

In [37]:
# save raw data in a pickle file
filename = 'test_pkl'
outfile = open(filename,'wb')
pickle.dump(txt,outfile)
outfile.close()

In [41]:
# reload pickled data
infile = open(filename,'rb')
new_txt = pickle.load(infile)
infile.close()

In [46]:
new_txt['payload']['parts'][0]

{'partId': '0',
 'mimeType': 'text/plain',
 'filename': '',
 'headers': [{'name': 'Content-Type',
   'value': 'text/plain; charset="utf-8"; format="fixed"'},
  {'name': 'Content-Transfer-Encoding', 'value': 'quoted-printable'}],
 'body': {'size': 2173,
  'data': 'aHR0cHM6Ly9rZG51Z2dldHMudXMxMi5saXN0LW1hbmFnZS5jb20vdHJhY2svY2xpY2s_dT00ZjI4OTFlYmIxNTViMjNmMTIwZWNlMGJkJmlkPTYyMjYzMzdiYjQmZT1iZmRmZDczYzE3DQoNCk9uY2UgdXBvbiBhIHRpbWUsIHRoZXJlIHdhcyBhIGNvbXBhbnkgdGhhdCBkcmVhbWVkIG9mIGNoYW5naW5nIHRoZSB3b3JsZCB3aXRoIG1hY2hpbmUgbGVhcm5pbmcuIFRoZSBicmlsbGlhbnQgZGF0YSB0ZWFtIGxhYmVsZWQgYXdheSBkYXkgYW5kIG5pZ2h0IGJ1dCBpdCB3YXMgbmV2ZXIgZW5vdWdoIHRvIGRhdGEuIFRoZSBtb2RlbHMganVzdCB3ZXJlbid0IHBlcmZvcm1pbmcgY29ycmVjdGx5LiBUaGV5IGxvbmdlZCBmb3IgdGhlIGJsaXNzZnVsIGRheXMgb2YgZGF0YSBhbmFseXNpcyBhbmQgaW5ub3ZhdGlvbi4NCg0KU291bmQgZmFtaWxpYXI_IFlvdSBtaWdodCBiZSByZWFkeSB0byBvdXRzb3VyY2UgeW91ciBkYXRhIGxhYmVsaW5nLg0KDQpZb3VyIGRyZWFtIG9mIGNoYW5naW5nIHRoZSB3b3JsZCBjYW4gY29tZSB0cnVlIHdpdGggYSBsaXR0bGUgaGVscCBmcm9tIHNvbWUg

## Check id query

In [6]:
filename = '/Users/markusmuller/python/projects/gmail-newsletter-db/scripts/id_query_17.09.2021'
infile = open(filename,'rb')
id_list = pickle.load(infile)
infile.close()

In [7]:
id_list 

{'messages': [{'id': '17bf36ad1f8ffa14', 'threadId': '17bf36ad1f8ffa14'},
  {'id': '17bf2d743a2a4a64', 'threadId': '17bf2d743a2a4a64'},
  {'id': '17bf2ba112b2990e', 'threadId': '17bf2ba112b2990e'},
  {'id': '17bf2b9a28a4e1cf', 'threadId': '17bf2b9a28a4e1cf'},
  {'id': '17bed324d52f36ee', 'threadId': '17bed324d52f36ee'},
  {'id': '17beb2ea7a06eb5e', 'threadId': '17beb2ea7a06eb5e'},
  {'id': '17be8bcdaa338b35', 'threadId': '17be8bcdaa338b35'},
  {'id': '17be5f07abd13e84', 'threadId': '17be5f07abd13e84'},
  {'id': '17be5ac70eaf3f1d', 'threadId': '17be5ac70eaf3f1d'},
  {'id': '17be368851c51489', 'threadId': '17be368851c51489'},
  {'id': '17be067784b98401', 'threadId': '17be067784b98401'},
  {'id': '17bdf62d721058f5', 'threadId': '17bdf62d721058f5'},
  {'id': '17bdafc60ea3954c', 'threadId': '17bdafc60ea3954c'},
  {'id': '17bd51b3bf685077', 'threadId': '17bd51b3bf685077'},
  {'id': '17bd4d4f6a8e5bd7', 'threadId': '17bd4d4f6a8e5bd7'},
  {'id': '17bd41ed7e3689c9', 'threadId': '17bd41ed7e3689c9

In the next request we can use 'nextPageToken' to get the next 500 mails

In [8]:
# should be 500 as we set max Result to 500
# to get more 
len(id_list['messages'])

500

In [9]:
filename = '/Users/markusmuller/python/projects/gmail-newsletter-db/scripts/id_query_17.09.2021_q2'
infile = open(filename,'rb')
id_list_2 = pickle.load(infile)
infile.close()

In [10]:
id_list_2

{'messages': [{'id': '17816701fa1d9c13', 'threadId': '17816701fa1d9c13'},
  {'id': '178131161b8442c3', 'threadId': '178131161b8442c3'},
  {'id': '1781126f55307a59', 'threadId': '1781126f55307a59'},
  {'id': '178076a41dea0508', 'threadId': '178076a41dea0508'},
  {'id': '178070e67f1b7a01', 'threadId': '178070e67f1b7a01'},
  {'id': '17802f10b90c9cc0', 'threadId': '17802f10b90c9cc0'},
  {'id': '177fe7088ffa657d', 'threadId': '177fe7088ffa657d'},
  {'id': '177fd57b08afc2c3', 'threadId': '177fd57b08afc2c3'},
  {'id': '177fd22793af68d6', 'threadId': '177fd22793af68d6'},
  {'id': '177fcb336f882499', 'threadId': '177fcb336f882499'},
  {'id': '177f7f9b3adf06c1', 'threadId': '177f7f9b3adf06c1'},
  {'id': '177f4a8bb7de2c33', 'threadId': '177f4a8bb7de2c33'},
  {'id': '177f490742fa4610', 'threadId': '177f490742fa4610'},
  {'id': '177f48f055fcf4f8', 'threadId': '177f48f055fcf4f8'},
  {'id': '177f42cf322b3b7a', 'threadId': '177f42cf322b3b7a'},
  {'id': '177f41fabe152c86', 'threadId': '177f41fabe152c86

In [12]:
filename = '/Users/markusmuller/python/projects/gmail-newsletter-db/scripts/id_query_17.09.2021_q3'
infile = open(filename,'rb')
id_list_3 = pickle.load(infile)
infile.close()

id_list_3

{'messages': [{'id': '17399a307f77cab8', 'threadId': '17399a307f77cab8'},
  {'id': '173974f11ed7790a', 'threadId': '173974f11ed7790a'},
  {'id': '17395b9d4a8ea12c', 'threadId': '17395b9d4a8ea12c'},
  {'id': '1738d5b273ab1830', 'threadId': '1738d5b273ab1830'},
  {'id': '1738533b8f2273bd', 'threadId': '1738533b8f2273bd'},
  {'id': '1738136480f2a6a9', 'threadId': '1738136480f2a6a9'},
  {'id': '1738034e8080a86d', 'threadId': '1738034e8080a86d'},
  {'id': '1737d164df70df8b', 'threadId': '1737d164df70df8b'},
  {'id': '1737ac6506ec325c', 'threadId': '1737ac6506ec325c'},
  {'id': '17377c63fea4b846', 'threadId': '17377c63fea4b846'},
  {'id': '1737649a940c39af', 'threadId': '1737649a940c39af'},
  {'id': '173737a8746577a9', 'threadId': '173737a8746577a9'},
  {'id': '17372e140111bdd8', 'threadId': '17372e140111bdd8'},
  {'id': '17371ae0375b20b8', 'threadId': '17371ae0375b20b8'},
  {'id': '1737099fe7305539', 'threadId': '1737099fe7305539'},
  {'id': '173706721b68dcef', 'threadId': '173706721b68dcef

as expected there is no nextPageToken, so we reached the end of our mailbox and we have no extracted every id 

## Newsletter: KDnuggets
- when you subscribe to KDnuggets you will get mails from two differrent mail addresses.
- one for events and news (editor1@kdnuggets.com) and one with top stories (mattmayo@kdnuggets.com)
- i want both but the first one is polluted with mails about events so i have to filter them later

In [34]:
search_str = "from:KDnuggets" 
filename = "KDnuggets_ids_"
date = datetime.date.today().strftime("%m.%d.%Y")
filename = filename + date + ".pkl"
print(filename)

KDnuggets_ids_09.30.2021.pkl


In [33]:
# paths
pathToToken = '/Users/markusmuller/python/projects/content-db/gmail/token.json'
data_path   = '/Users/markusmuller/python/projects/content-db/gmail/data'

In [37]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = Credentials.from_authorized_user_file(pathToToken, SCOPES)

# connect to Gmail api
service = build('gmail', 'v1', credentials=creds)

# request a list of all the messages
result = service.users().messages().list(userId='markus.mueller.ds@gmail.com', q=search_str, maxResults=500).execute()

In [38]:
result

{'messages': [{'id': '17c313783d7e09f1', 'threadId': '17c313783d7e09f1'},
  {'id': '17c30c98e39c3de7', 'threadId': '17c30c98e39c3de7'},
  {'id': '17c2b7d2843852d9', 'threadId': '17c2b7d2843852d9'},
  {'id': '17c25e3aa627293e', 'threadId': '17c25e3aa627293e'},
  {'id': '17c2311129d967a5', 'threadId': '17c2311129d967a5'},
  {'id': '17c1c3c531e08e3e', 'threadId': '17c1c3c531e08e3e'},
  {'id': '17c16e39b6a5d42e', 'threadId': '17c16e39b6a5d42e'},
  {'id': '17c11c78f016205d', 'threadId': '17c11c78f016205d'},
  {'id': '17c0d04c01474fa8', 'threadId': '17c0d04c01474fa8'},
  {'id': '17c0c9b99548525d', 'threadId': '17c0c9b99548525d'},
  {'id': '17c0796a03484489', 'threadId': '17c0796a03484489'},
  {'id': '17c023c69e0bc049', 'threadId': '17c023c69e0bc049'},
  {'id': '17bf839a7969c51f', 'threadId': '17bf839a7969c51f'},
  {'id': '17bf2d743a2a4a64', 'threadId': '17bf2d743a2a4a64'},
  {'id': '17bed324d52f36ee', 'threadId': '17bed324d52f36ee'},
  {'id': '17be8bcdaa338b35', 'threadId': '17be8bcdaa338b35

In [39]:
# save id as pkl
with open(os.path.join(data_path, filename), 'wb') as f:
    pickle.dump(result, f)

In [40]:
# for loop to get id 
for id in result['messages']:
    print(id['id'])

17c313783d7e09f1
17c30c98e39c3de7
17c2b7d2843852d9
17c25e3aa627293e
17c2311129d967a5
17c1c3c531e08e3e
17c16e39b6a5d42e
17c11c78f016205d
17c0d04c01474fa8
17c0c9b99548525d
17c0796a03484489
17c023c69e0bc049
17bf839a7969c51f
17bf2d743a2a4a64
17bed324d52f36ee
17be8bcdaa338b35
17be368851c51489
17bdafc60ea3954c
17bd41ed7e3689c9
17bcef38c960fc3b
17bc92883487c59a
17bc4a99883075e1
17bc47ca455dea43
17bbf853dfebce27
17bba2e6ee21ba8a
17bab5189de5ff8a
17ba5c0f63e5d7a6
17ba598e729bf32b
17ba0a53c538f36a
17b9d3e197b14d4f
17b8c0b2c93af7d1
17b86ab90b36ee35
17b7c8c29c863548
17b7c536cd4056ff
17b77a030916434b
17b7213fcd2d544f
17b6f44da0eae264
17b68035bfd5b604
17b63061251c287e
17b5da7ddeb42a02
17b58c81ef295ea4
17b587ef70b09641
17b5390c85ea26ff
17b4e02fc2048dcc
17b43f177da6552a
17b40785fb74b467
17b39dc773a967cd
17b348465b5af49d
17b29f5b10ba35ef
17b1fd83cfc3564e
17b1a7b722cc5175
17b14e8515e8eae5
17b1077078112e18
17b1036fbf83d6e5
17b0b46c098ede10
17b0131a29d55b1e
17afc3e0c305615a
17af604c55a21738
17af0dc83593c6

In [45]:
# get data
txt = service.users().messages().get(userId='me', id='1736134e69aec8df').execute()

In [46]:
# payload['headers'] is a list of dicts
payload = txt['payload']
for d in payload['headers']:
    if d['name'] == 'Date':
        date = d['value']
    if d['name'] == 'From':
        sender = d['value']
    if d['name'] == 'Subject':
        subject = d['value']
    
print('date: ', date)
print('from: ', sender)
print('subject: ', subject)

date:  Sat, 18 Jul 2020 09:14:49 +0000
from:  Matthew Mayo <mattmayo@kdnuggets.com>
subject:  The Bitter Lesson of Machine Learning; Free MIT Courses on Calculus: The Key to Understanding Deep Learning


now that we know that we got every id let us get the email data

In [50]:
list_mail = []
# create a counter know the position if the code breaks
counter = 0
# for loop to get id 
for id in result['messages']:
    mail = service.users().messages().get(userId='me', id=id['id']).execute()
    list_mail.append(mail)
    counter = counter + 1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


thats workes now I have every mail from KDNuggets

In [51]:
len(list_mail)

349

In [58]:
filename = "KDnuggets_mails_"
date = datetime.date.today().strftime("%m.%d.%Y")
filename = filename + date + ".pkl"
print(filename)

KDnuggets_mails_09.30.2021.pkl


In [59]:
# save data
with open(os.path.join(data_path, filename), 'wb') as f:
    pickle.dump(list_mail, f)

### Create parser for KDnuggest
- filter out unrelevant mails
    - i cant use the title since there is no indication if a mail relevant or not
    - but i can use snipptes if it is relevant it will state the following in the snippet:
        - "News" or "Top Stories"
1. build a pandas dataframe with the relevant data
2. filter out mails from mattmayo@kdnuggets.com

In [61]:
list_mail[0].keys()

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])

In [62]:
list_mail[0]['id']

'17c313783d7e09f1'

In [63]:
list_mail[0]['snippet']

'Data Management 101 on Databricks How do organizations avoid a data management mess and uplevel their process to more efficiently serve downstream analytics, data science and machine learning? Get the'

In [66]:
list_mail[5]['snippet']

'Also: The Machine &amp; Deep Learning Compendium Open Book; Easy SQL in Native Python KDnuggets KDnuggets™ Top Stories, Sep 25, 2021 Please consider submitting an original blog to KDnuggets! If it is'

In [132]:
# create DF with relevant data
id_list = []
date_list = []
from_list = []
subject_list = []
snippet_list = []
content_list = []

for counter, id_ in enumerate(result['messages']):
    id_ = id_['id']

    payload = list_mail[counter]['payload']
    for d in payload['headers']:
        if d['name'] == 'Date':
            date = d['value']
        if d['name'] == 'From':
            sender = d['value']
        if d['name'] == 'Subject':
            subject = d['value']

    # there are some mails that dont have a parts section
    if "parts" in list_mail[counter]['payload'].keys():
        parts = payload.get('parts')[0]
        data = parts['body']['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)
        soup = BeautifulSoup(decoded_data , "lxml")
        body = soup.body()
        body_str = str(body)
    else:
        data = payload['body']['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)
        soup = BeautifulSoup(decoded_data , "lxml")
        body = soup.body()
        body_str = str(body)


    id_list.append(id_)
    date_list.append(date)
    from_list.append(sender)
    subject_list.append(subject)
    snippet_list.append(list_mail[counter]['snippet'])
    content_list.append(body_str)

    
#print('id: ', list_mail[0]['id'])
#print('date: ', date)
#print('from: ', sender)
#print('subject: ', subject)
#print('snippet: ', list_mail[0]['snippet'])
#print('content: ', body_str)

In [133]:
len(snippet_list)

349

In [131]:
print(date_list[348])

Sat, 18 Jul 2020 09:14:49 +0000


In [134]:
print(content_list[348])

[<p>Also: A Layman's Guide to Data Science. Part 3: Data Science Workflow; Exploratory Data Analysis on Steroids

https://www.kdnuggets.com/index.html

KDnuggets™ Top Stories, Jul 18, 2020 (https://www.kdnuggets.com/news/top-stories.html)


** Featured Story
------------------------------------------------------------
* The Bitter Lesson of Machine Learning (https://www.kdnuggets.com/2020/07/bitter-lesson-machine-learning.html)

https://www.kdnuggets.com/2020/07/bitter-lesson-machine-learning.html


** Most Popular Last Week
------------------------------------------------------------

1. A Layman's Guide to Data Science. Part 3: Data Science Workflow (https://www.kdnuggets.com/2020/07/laymans-guide-data-science-workflow.html)
2. Free MIT Courses on Calculus: The Key to Understanding Deep Learning (https://www.kdnuggets.com/2020/07/free-mit-courses-calculus-key-deep-learning.html)
3. Exploratory Data Analysis on Steroids (https://www.kdnuggets.com/2020/07/exploratory-data-analysis-ster